In [7]:
import yaml, os
from llama_index.llms import AzureOpenAI, OpenAI
from llama_index.llm_predictor import LLMPredictor
from llama_index import set_global_service_context
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.text_splitter import TokenTextSplitter
from llama_index.indices import ZillizCloudPipelineIndex
from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters
from llama_index import ServiceContext, load_index_from_storage, StorageContext

## Architecture
![alt text](https://miro.medium.com/v2/resize:fit:720/format:webp/1*erE3lOZr_WSBaf6NmTP6hQ.jpeg)

## Configure LLMs / VDB Clouds

In [2]:
with open('/Users/1zuu/Desktop/LLM RESEARCH/LLM in Cloud/cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

# os.environ['OPENAI_API_KEY'] = credentials['OPENAI_API_KEY']

In [3]:
llm_flag = 'DIRECT'

embedding_llm = HuggingFaceEmbedding(
                                    model_name="BAAI/bge-large-en-v1.5",
                                    device='mps'
                                    )

if llm_flag == 'AZURE':
    llm=AzureOpenAI(
                    model=credentials['AZURE_ENGINE'],
                    api_key=credentials['AZURE_OPENAI_API_KEY'],
                    deployment_name=credentials['AZURE_DEPLOYMENT_ID'],
                    api_version=credentials['AZURE_OPENAI_API_VERSION'],
                    azure_endpoint=credentials['AZURE_OPENAI_API_BASE'],
                    temperature=0.3
                    )
    
    chat_llm = LLMPredictor(llm)
else:
    chat_llm = OpenAI(
                    api_key=credentials['OPENAI_API_KEY'],
                    temperature=0.3
                    )

text_splitter = TokenTextSplitter(
                                separator=" ",
                                chunk_size=1024,
                                chunk_overlap=20,
                                backup_separators=["\n"]
                                )

if llm_flag == 'AZURE':
    service_context = ServiceContext.from_defaults(
                                                    text_splitter=text_splitter,
                                                    # prompt_helper=prompt_helper,
                                                    embed_model=embedding_llm,
                                                    llm_predictor=chat_llm
                                                    )
else:
    service_context = ServiceContext.from_defaults(
                                                    text_splitter=text_splitter,
                                                    # prompt_helper=prompt_helper,
                                                    embed_model=embedding_llm,
                                                    llm=chat_llm
                                                    )

set_global_service_context(service_context)

In [5]:
zcp_index = ZillizCloudPipelineIndex(
                                    project_id=credentials['ZILLIZ_PROJECT_ID'],
                                    cluster_id=credentials['ZILLIZ_CLUSTER_ID'],
                                    token=credentials['ZILLIZ_API_KEY'],
                                    )

try:
    zcp_index.create_pipelines(
                                metadata_schema={
                                                "user_id": "VarChar", 
                                                "version": "VarChar"
                                                }
                                )
    print('Index created')
except:
    print('Index already exists')
    pass

Index already exists


## Ingest Documents

In [6]:
# user1 ingests a document, it is technical documentation for v2.3 version. 
zcp_index.insert_doc_url(
                        url="https://publicdataset.zillizcloud.com/milvus_doc.md",
                        metadata={
                                "user_id": "user1", 
                                "version": "2.3"
                                },
                        )

# user2 ingests a document, it is technical documentation for v2.2 version. 
zcp_index.insert_doc_url(
                        url="https://publicdataset.zillizcloud.com/milvus_doc_22.md",
                        metadata={
                                "user_id": "user2", 
                                "version": "2.2"
                                },
                        )

{'token_usage': 984, 'doc_name': 'milvus_doc_22.md', 'num_chunks': 7}

## Query

In [9]:
"""
The chatbot will only answer with the retrieved information from user1's documents
"""

query_engine_for_user1 = zcp_index.as_query_engine(
                                                    search_top_k=3,
                                                    filters=MetadataFilters(
                                                                            filters=[
                                                                                    ExactMatchFilter(
                                                                                                    key="user_id", 
                                                                                                    value="user1"
                                                                                                    )
                                                                                    ]  # The query would only search from documents of user1.
                                                                            ),
                                                    output_metadata=[
                                                                    "user_id", 
                                                                    "version"
                                                                    ], # output these tags together with document text
                                                    )

question = "Can users delete entities by complex boolean expressions?"
answer = query_engine_for_user1.query(question)
print(answer)

Yes, users can delete entities by complex boolean expressions.
